In [66]:
from conformal_sparsemax.classifier import CNN,get_data, evaluate
from sklearn.metrics import f1_score
import torch
import numpy as np
import pickle
from entmax.losses import SparsemaxLoss, Entmax15Loss

In [67]:
loss = 'sparsemax' #sparsemax, softmax or entmax15
dataset='CIFAR100' #CIFAR100 or MNIST

In [68]:
model = CNN(loss)
_,_, test_dataloader, cal_dataloader = get_data(0.2,16)
model.load_state_dict(torch.load(f'models/{dataset}_{loss}.pth'))
if loss == 'sparsemax':
    criterion = SparsemaxLoss()
elif loss == 'entmax15':
    criterion = Entmax15Loss()
elif loss == 'softmax':
    criterion = torch.nn.NLLLoss()

Files already downloaded and verified
Files already downloaded and verified


In [69]:
test_proba, test_pred, test_true, test_loss = evaluate(
                                                    model,
                                                    test_dataloader,
                                                    criterion)

test_f1 = f1_score(test_pred, test_true, average='weighted')

print(f'Test loss: {test_loss:.3f}')
print(f'Test f1: {test_f1:.3f}')

Test loss: 0.441
Test f1: 0.304


In [70]:
print('type:', type(test_proba))
print('pred_proba shape:',test_proba.shape)
print('pred_labels shape:',test_pred.shape)
print('true_labels shape:',test_true.shape)

type: <class 'numpy.ndarray'>
pred_proba shape: (10000, 100)
pred_labels shape: (10000,)
true_labels shape: (10000,)


In [71]:
((test_proba==0).sum(axis=1)/100).mean()

0.917632

In [72]:
cal_proba, cal_pred, cal_true, cal_loss = evaluate(
                                                    model,
                                                    cal_dataloader,
                                                    criterion)

cal_f1 = f1_score(cal_pred, cal_true, average='weighted')

print(f'Calibration loss: {cal_loss:.3f}')
print(f'Calibration f1: {cal_f1:.3f}')

Calibration loss: 0.377
Calibration f1: 0.455


In [73]:
#One Hot Encoding
test_true_enc = np.zeros((test_true.size, test_true.max()+1), dtype=int)
test_true_enc[np.arange(test_true.size),test_true] = 1

cal_true_enc = np.zeros((cal_true.size, cal_true.max()+1), dtype=int)
cal_true_enc[np.arange(cal_true.size),cal_true] = 1

In [74]:
predictions = {'test':{'proba':test_proba,'true':test_true_enc},
 'cal':{'proba':cal_proba,'true':cal_true_enc}}

In [80]:
for dataset_type in ['cal','test']:
    for y in ['proba','true']:
        with open(f'predictions/{dataset}_{loss}_{dataset_type}_{y}.pickle', 'wb') as f:
            pickle.dump(predictions[dataset_type][y], f)

In [82]:
with open('predictions/CIFAR100_sparsemax_test_true.pickle', 'rb') as f:
    test_true = pickle.load(f)